In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def clear_data(string1):
    # Remplacer "Nets" par "\nNets", "Angels" par "\nAngels", etc.
    output_string = re.sub(r'([a-zA-Z]+)([A-Z][a-z]+)', r'\1,\2', string1)

    # Supprimer les balises de note [note 10], [note 12], [note 14]
    output_string = re.sub(r'\[note \d+\]', '', output_string)

    utput_string = re.sub(r'[^a-zA-Z\s]+', '', output_string)
    
    # Convertir la chaîne en liste
    output_list = output_string.split(',')

    return output_list


def clear_nba_data(string):
    result= re.sub(r'\s*\*?\s*\(\d+\)$', '', string)
    return result


def get_area(team,mlb_cities):
    for index, row in mlb_cities.iterrows():
        if team in index:
            return row['Metropolitan area']
        
def get_ratio(out):
    for item in out:
        if item['Area'] == 'Atlanta':
            ratio_value = item['Ratio']
            break
    return ratio_value


def get_nba_data():
    return out_df

def mlb_correlation(): 
    population_by_region = []  # pass in metropolitan area population from cities
    win_loss_by_region = []  # pass in win/loss ratio from nhldfin the same order as cities["Metropolitan area"]
    mlb_df= pd.read_csv("assets/mlb.csv")
    cities = pd.read_html("assets/wikipedia_data.html")[1]
    #print(cities)  

    cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
    cities.rename(columns={"Population (2016 est.)[8]": "Population"},inplace=True)
    mlb_df= mlb_df[mlb_df['year'] == 2018]  # get only 2018 stats no need of dropping rows
    population = cities[['Metropolitan area', 'Population']]

    #print(population)
    #population = population.set_index('Metropolitan area')

    #cities=cities['MLB'].replace("",np.nan).replace("—",np.nan).dropna()

    #print(cities)
    #print(mlb_df)
    #####"cities['team'] = cities['MLB'].apply(lambda x: clear_data(x))
    #cities['MLB'] = cities['MLB'].apply(lambda x: clear_data(x))
 
    new_rows = []
    row=0
    WLratio=0

    cities['W-L%']=0
    citiestemp=cities.copy()
    for index, row in citiestemp.iterrows():
        cleaned_values = clear_data(row['MLB'])
        if len(cleaned_values) > 1:
            for valueTeam in cleaned_values[0:]:
                ville=row['Metropolitan area']
                pop=population.loc[population['Metropolitan area']==ville,'Population'].iloc[0]
                
                WLratio=mlb_df.loc[mlb_df['team'].str.contains(valueTeam),'W-L%'].iloc[0]
                print("WLratioooooooo"+str(WLratio))

                valeurs_correspondantes = cities.loc[cities['MLB'].str.contains(valueTeam), 'Metropolitan area'].iloc[0]    
                print('WLratio '+str(WLratio)+' '+str(valeurs_correspondantes)+' '+str(valueTeam))                                            
                new_rows.append({'Metropolitan area': ville, 'MLB': valueTeam, 'Population': pop, 'W-L%': str(WLratio)})
            cities=cities.drop(index)

            
        
        if len(cleaned_values) == 1:
            team=cleaned_values[0]
            #WLratio=mlb_df.loc[mlb_df['team'].str.contains('Mets'),'W-L%'].iloc[0]
            cities.loc[index,'MLB']=team
            #cities.loc[index,'W-L%'] = WLratio


    # Étendre le DataFrame avec les nouvelles lignes
    cities = cities.append(new_rows, ignore_index=True)

    print('cities')
    #print(cities)
    
    #cities['MLB'] = cities['MLB'].apply(lambda x: clear_data(x)[0])



    
    mlb_cities = cities[['Metropolitan area', 'MLB','Population','W-L%']].set_index('MLB')

    mlb_cities = mlb_cities.drop(['—', ''], axis=0)
    #mlb_cities['W-L%'] = mlb_cities['MLB'].apply(lambda x: mlb_df.loc[mlb_df['team'].str.contains(x),'W-L%'].iloc[0])
    #mlb_cities.reindex()
    for index, row in  mlb_cities.iterrows():
        WLratio=mlb_df.loc[mlb_df['team'].str.contains(index),'W-L%'].iloc[0]
        mlb_cities.loc[index,'W-L%'] = WLratio
    print(mlb_cities)
   
    #print(mlb_df) 
    #mlb_df['team'] = mlb_df['team'].apply(lambda x: clear_nba_data(x))
   
    #mlb_df['team'] = mlb_df['team'].apply(lambda x: x.split(" ")[-1])
    # print(mlb_df['team'])
      
    #mlb_df['city'] = mlb_df['team'].apply(lambda x: get_area(x,mlb_cities))
    #print(mlb_df['city'])
    #mlb_df['area']=cities['Metropolitan area']
    #print(mlb_df)   

    out = []
    ratio=0
    for group, frame in mlb_cities.groupby('Metropolitan area'):
        #total_wins = np.sum(pd.to_numeric(frame['W']))
        #total_losses = np.sum(pd.to_numeric(frame['L']))
        WL=np.mean(pd.to_numeric(frame['W-L%']))
        
        #total_matches = total_wins + total_losses
        ratio = WL#(total_wins / total_matches)
        
        rv=next((item['Ratio'] for item in out if item['City'] == group), 0)
        #print(rv)
        out_dict = {
            'City': group,
            'Ratio': next((item['Ratio'] for item in out if item['City'] == group), 0) + ratio
        }
        #print(next((item['Ratio'] for item in out if item['Area'] == group), 0))
        out.append(out_dict)
    #print(out)
    new_df= pd.DataFrame(out)
    new_df= new_df.set_index('City')
    #print(new_df)
    #print(population)
    
    out_df= pd.merge(new_df, population, how="inner", left_index=True, right_index=True)
    out_df['Population'] = pd.to_numeric(out_df['Population'])
    print(out_df['Population'].tolist())
    
    mlb_cities= mlb_cities.astype({'Population' : int, 'W-L%': float})
    population_by_region = mlb_cities['Population'] #out_df['Population'].tolist()
    win_loss_by_region = mlb_cities['W-L%'] #out_df['Ratio'].to_list()
    corr = stats.pearsonr(population_by_region, win_loss_by_region)[0]
    return corr


    
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from mlbdfin the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)


mlb_correlation()

WLratioooooooo0.617
WLratio 0.617 New York City Yankees
WLratioooooooo0.475
WLratio 0.475 New York City Mets
WLratioooooooo0.564
WLratio 0.564 Los Angeles Dodgers
WLratioooooooo0.494
WLratio 0.494 Los Angeles Angels
WLratioooooooo0.451
WLratio 0.451 San Francisco Bay Area Giants
WLratioooooooo0.599
WLratio 0.599 San Francisco Bay Area Athletics
WLratioooooooo0.583
WLratio 0.583 Chicago Cubs
WLratioooooooo0.3829999999999999
WLratio 0.3829999999999999 Chicago White Sox
cities
                   Metropolitan area Population   W-L%
MLB                                                   
Rangers            Dallas–Fort Worth    7233323  0.414
Nationals           Washington, D.C.    6131977  0.506
Phillies                Philadelphia    6070500  0.494
Red Sox                       Boston    4794447  0.667
Twins         Minneapolis–Saint Paul    3551036  0.481
Rockies                       Denver    2853077  0.558
Marlins        Miami–Fort Lauderdale    6066387  0.391
Diamondbacks              

C:\Users\Pierre\AppData\Local\Temp\ipykernel_16488\487670272.py:95: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cities = cities.append(new_rows, ignore_index=True)


0.17248163898939659